In [1]:
import tensorflow as tf
from keras import layers
from cnn_classifier import *

2023-03-02 13:47:07.297012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_shape = (128,128,3)
in_channels = 3
out_classes = 4
filters = [8,8,16,16,32,32,48,64]
kernel_size = [3,3,3,3,3,3,3,3]
strides = [2,1,2,1,2,1,1,1]
padding='valid'
activation='relu'
downsampling_mode='stride'
flatten=False
fc=[128,16]
dropout = 0.5
spatial_dropout = 0.2
lambda_l2 = 0.0005
lrate = 0.001
loss = 'sparse_categorical_crossentropy'
metrics = None

In [3]:
model = create_cnn_classifier_network(input_shape, in_channels, out_classes, filters, kernel_size, strides, padding=padding, activation=activation, downsampling_mode=downsampling_mode, flatten=flatten, fc=fc)
model.summary()

2023-03-02 13:47:14.396049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 63, 8)         224       
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 8)         584       
                                                                 
 conv2d_2 (Conv2D)           (None, 30, 30, 16)        1168      
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 16)        2320      
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 conv2d_6 (Conv2D)           (None, 9, 9, 48)          1

In [4]:
x = tf.random.normal((1,128,128,3))
y = model(x)
print(y)

tf.Tensor([[0.25671077 0.25501975 0.24265423 0.2456152 ]], shape=(1, 4), dtype=float32)
